**Importing the necessary modules**


In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

**Connecting to the Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
IMAGE_SIZE = [224, 224]

**Loading the dataset's from the directory**

In [ ]:
train_directory= '/content/drive/My Drive/Colab Notebooks/plant_1/TRAIN_NEW'
test_directory= '/content/drive/My Drive/Colab Notebooks/plant_1/test'
val_directory= '/content/drive/My Drive/Colab Notebooks/plant_1/valid'

**Freezing Pre-trained VGG16 Model Layers in Keras**

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

**Adding Custom Layers to Pre-trained VGG16 Model in Keras**

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

**Compiling the Customized VGG16 Model in Keras.**

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

**Preparing Image Data Generators for Training and Testing in Keras.**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_directory,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_directory,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

print(len(training_set))
print(len(test_set))

**Training the Customized VGG16 Model on Image Data in Keras.**

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

**Plotting Training and Validation Losses.**

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

**Plotting Training and Validation Accuracies.**

In [ ]:
# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

**Saving the Trained Model Weights**

In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/plant_1/Model_weights.h5')